# FIN411X – Stats Analysis Notebook
ChatGPT survey (AI literacy & metacognition) + portfolio performance

This notebook:
1. Loads `summary_all_portfolios.csv` (portfolio performance).
2. Loads `chatgpt_survey_raw_id.csv` (survey responses **with** `Student_id`).
3. Merges them by student ID (S01–S30).
4. Builds three scales:
   - B_scale: Learning & confidence (B1–B6)
   - C_scale: Metacognitive strategies (C1–C6)
   - D_scale: AI literacy / critical awareness (D1–D6)
5. Computes Cronbach's alpha for each scale.
6. Computes descriptive statistics for performance + scales.
7. Computes a correlation matrix and selected r, p-values.

**Instructions:**
- Upload `summary_all_portfolios.csv` and `chatgpt_survey_raw_id.csv` into Colab (left panel → Upload).
- Then run the cells from top to bottom.


In [ ]:
# Install required libraries (only needed once)
!pip -q install scipy


In [ ]:
# Imports
import pandas as pd
import numpy as np
from scipy import stats

print("Libraries imported.")


## 1. Load the two datasets

- `summary_all_portfolios.csv` = portfolio performance (alpha, beta, return, etc.)
- `chatgpt_survey_raw_id.csv` = survey responses including `Student_id`.


In [ ]:
# Load portfolio performance summary
summary = pd.read_csv("summary_all_portfolios.csv")

# Load ChatGPT survey with Student_id codes (S01–S30)
survey = pd.read_csv("chatgpt_survey_raw_id.csv")

print("Summary shape:", summary.shape)
print("Survey shape :", survey.shape)

display(summary.head())
display(survey.head())


## 2. Merge on student ID

We join the two files using `student_id` (from the portfolio summary)
and `Student_id` (from the survey). Make sure the codes (S01, S02, ...) match.


In [ ]:
# Strip any whitespace from IDs, just in case
summary["student_id"] = summary["student_id"].astype(str).str.strip()
survey["Student_id"]  = survey["Student_id"].astype(str).str.strip()

# Inner join = keep only students present in BOTH files
df = summary.merge(survey, left_on="student_id", right_on="Student_id", how="inner")

print("Merged shape:", df.shape)
df.head()


## 3. Identify B, C, and D scale items

The Google Forms export keeps the question texts as column names.
In this dataset, the indices for the B/C/D items are:
- B1–B6: columns 8–13
- C1–C6: columns 14–19
- D1–D6: columns 20–25

The cell below prints these column names so you can confirm.


In [ ]:
# Inspect all survey column indices and names (optional, for checking)
print("Survey columns with indices:\n")
for i, col in enumerate(survey.columns):
    print(i, "->", col[:80])

# Select columns for the three Likert scales based on their positions
B_cols = survey.columns[8:14]   # Section B: learning & confidence (6 items)
C_cols = survey.columns[14:20]  # Section C: metacognitive strategies (6 items)
D_cols = survey.columns[20:26]  # Section D: AI literacy / critical awareness (6 items)

print("\nB-scale items:\n", B_cols)
print("\nC-scale items:\n", C_cols)
print("\nD-scale items:\n", D_cols)


## 4. Cronbach's alpha helper function

We define a function to compute Cronbach's alpha for a set of Likert items.


In [ ]:
def cronbach_alpha(items_df: pd.DataFrame) -> float:
    """Compute Cronbach's alpha for a set of Likert items (columns)."""
    items = items_df.dropna()
    k = items.shape[1]
    if k < 2:
        return np.nan

    item_vars = items.var(axis=0, ddof=1)
    total_scores = items.sum(axis=1)
    total_var = total_scores.var(ddof=1)

    alpha = (k / (k - 1.0)) * (1.0 - item_vars.sum() / total_var)
    return float(alpha)

print("Alpha function ready.")


## 5. Build B, C, D scales and compute reliability

- `B_scale` = mean of Section B items (learning & confidence)
- `C_scale` = mean of Section C items (metacognitive strategies)
- `D_scale` = mean of Section D items (AI literacy / critical awareness)


In [ ]:
# Add scale scores to the merged dataframe
df["B_scale"] = df[B_cols].mean(axis=1)
df["C_scale"] = df[C_cols].mean(axis=1)
df["D_scale"] = df[D_cols].mean(axis=1)

# Compute Cronbach's alpha for each scale (using the survey data)
alpha_B = cronbach_alpha(df[B_cols])
alpha_C = cronbach_alpha(df[C_cols])
alpha_D = cronbach_alpha(df[D_cols])

print(f"Scale B (learning/confidence) alpha: {alpha_B:.3f}")
print(f"Scale C (metacognition) alpha      : {alpha_C:.3f}")
print(f"Scale D (AI literacy) alpha        : {alpha_D:.3f}")


## 6. Descriptive statistics (performance + scales)

We describe:
- Portfolio performance: `total_return_pct`, `alpha_annual`, `beta`
- Scales: `B_scale`, `C_scale`, `D_scale`.


In [ ]:
perf_vars = ["total_return_pct", "alpha_annual", "beta"]
vars_for_desc = perf_vars + ["B_scale", "C_scale", "D_scale"]

desc = df[vars_for_desc].describe().T
desc = desc[["mean", "std", "min", "25%", "50%", "75%", "max"]]

print("Descriptive statistics (performance + scales):\n")
desc.round(3)


## 7. Correlation matrix

We compute Pearson correlations between performance metrics and the B/C/D scales.


In [ ]:
corr_vars = ["total_return_pct", "alpha_annual", "beta",
             "B_scale", "C_scale", "D_scale"]

corr_matrix = df[corr_vars].corr()
print("Correlation matrix:\n")
corr_matrix.round(3)


## 8. Selected correlations with r and p-values

We inspect a few key relationships between AI-related scales and performance:
- B_scale, C_scale, D_scale with `total_return_pct`
- B_scale, C_scale, D_scale with `alpha_annual`.


In [ ]:
pairs = [
    ("B_scale", "total_return_pct"),
    ("C_scale", "total_return_pct"),
    ("D_scale", "total_return_pct"),
    ("B_scale", "alpha_annual"),
    ("C_scale", "alpha_annual"),
    ("D_scale", "alpha_annual"),
]

print("Pairwise correlations (r, p):\n")
for x, y in pairs:
    r, p = stats.pearsonr(df[x], df[y])
    print(f"{x} ~ {y}: r = {r:.3f}, p = {p:.3f}")


## 9. Save outputs (optional)

You can save the descriptive statistics and correlation matrix as CSV files to include
in your replication package or manuscript.


In [ ]:
# Save outputs
desc.to_csv("fin411x_descriptives.csv")
corr_matrix.to_csv("fin411x_correlations.csv")

print("Saved: fin411x_descriptives.csv and fin411x_correlations.csv")
